In [1]:
import json
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from collections import defaultdict

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sys import getsizeof
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/felicityzhao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/felicityzhao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
inverted_index = defaultdict(dict)
STOP_WORDS = set(stopwords.words('english'))

In [4]:
def lemma(word):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(word)

In [5]:
#Read json file
news_json = open("title_links.json")
news = json.load(news_json)
for k, v in news.items():
    url = "https://www.ucihealth.org" + v
    html = urlopen(url)
    soup = BeautifulSoup(html)
    k = k.replace('\u', ' ')
    
    #Fetch Contents
    contents = soup.find_all('p')
    article = []
    for c in contents:
        elem = c.get_text(strip=True)
        elem = elem.replace('\xa0', ' ')
        if elem != '':
            article.append(elem)
    
    #all_text to lower case, split by nonalphanumeric characters
    for text in article:
        text = re.sub(r'[^a-z0-9]', ' ', text.lower())
        
        for word in text.split():
            if len(word) > 2:
                word=lemma(word)
                if word not in STOP_WORDS:
                    if word not in inverted_index:
                        word_dict = {k:1}
                        inverted_index[word] = word_dict
                    else:
                        if k in inverted_index[word]:
                            inverted_index[word][k] += 1
                        else:
                            word_dict = {k:1}
                            inverted_index[word].update(word_dict)



SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape (<ipython-input-5-a688eeca6cb5>, line 8)

In [ ]:
#sort frequencies for each key words
final_dict = defaultdict(list)
for k,v in inverted_index.items():
    sorted_tuples = sorted(v.items(), key=lambda item: -item[1])
    final_dict[k] = sorted_tuples
    

In [ ]:
with open("inverted_index_words.json", "w") as outfile:  
    json.dump(final_dict, outfile)